In [1]:
from tensorflow.python.keras.layers import Input,Dense,Reshape,Lambda,RNN,Concatenate,StackedRNNCells
from tensorflow.python.keras.models import Model,load_model
from tensorflow.python.keras.utils import multi_gpu_model
from tensorflow.python.keras.optimizers import Adam,SGD
from tensorflow.python.keras.callbacks import LearningRateScheduler,Callback,ModelCheckpoint,TensorBoard

import tensorflow as tf
import datetime
import os
import numpy as np
import pickle

%env PYTHONHASHSEED=0
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
os.environ["CUDA_VISIBLE_DEVICES"]="1"

/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/

env: PYTHONHASHSEED=0


/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site

In [2]:
from sklearn.model_selection import train_test_split
#features
n2v_start = np.load("start_emb_128.npy")
n2v_end = np.load("end_emb_128.npy")
lanes = np.load("lanes.npy")
oneway = np.load("oneway.npy")
roadtype = np.load("highway.npy")
maxspeed = np.load("maxspeed.npy")
nodetype = np.load("nodetype.npy")
lanes_df = np.load("lanes_df.npy")
#target
meanspeed = np.load("meanspeed_only.npy")
z_score = np.load('z-score.npy')
congestion = np.load('congestion.npy')

#split
n2v_start_train, n2v_start_test, n2v_end_train, n2v_end_test, lanes_train, lanes_test, oneway_train, oneway_test, roadtype_train, roadtype_test, maxspeed_train, maxspeed_test, nodetype_train, nodetype_test, lanes_df_train, lanes_df_test, meanspeed_train, meanspeed_test, z_score_train, z_score_test,congestion_train,congestion_test = train_test_split(n2v_start, n2v_end, lanes,oneway,roadtype ,maxspeed,nodetype,lanes_df,meanspeed,z_score,congestion)

del n2v_start
del n2v_end
del lanes
del oneway
del roadtype
del maxspeed
del nodetype
del lanes_df
del meanspeed
del z_score
del congestion

In [3]:
embinplist = list()
n2vlist = list()
inplist = list()

n2v_start_inp = Input((128,))
n2v_start_reg = Dense(128,activation = "relu", use_bias = True)(n2v_start_inp)
inplist.append(n2v_start_inp)
n2vlist.append(n2v_start_reg)

n2v_end_inp = Input((128,))
n2v_end_reg = Dense(128,activation = "relu", use_bias = True)(n2v_end_inp)
inplist.append(n2v_end_inp)
n2vlist.append(n2v_end_reg)

lanes_inp = Input((1,))
lanes_reg= Dense(1,activation = "relu", use_bias = True)(lanes_inp)
inplist.append(lanes_inp)
embinplist.append(lanes_reg)

oneway_inp = Input((2,))
oneway_reg= Dense(2,activation = "relu", use_bias = True)(oneway_inp)
inplist.append(oneway_inp)
embinplist.append(oneway_reg)

roadtype_inp = Input((13,))
roadtype_reg= Dense(8,activation = "relu", use_bias = True)(roadtype_inp)
inplist.append(roadtype_inp)
embinplist.append(roadtype_reg)

maxspeed_inp = Input((1,))
maxspeed_reg= Dense(1,activation = "relu", use_bias = True)(maxspeed_inp)
inplist.append(maxspeed_inp)
embinplist.append(maxspeed_reg)

nodetype_inp = Input((11,))
nodetype_reg= Dense(8,activation = "relu", use_bias = True)(nodetype_inp)
inplist.append(nodetype_inp)
embinplist.append(nodetype_reg)

lanes_df_inp = Input((1,))
lanes_df_reg= Dense(1,activation = "relu", use_bias = True)(lanes_df_inp)
inplist.append(lanes_df_inp)
embinplist.append(lanes_df_reg)

n2v = Concatenate()(n2vlist)
n2v_reg = Dense(64,activation = "relu",use_bias = True)(n2v)
embinplist.append(n2v_reg)

combine = Concatenate()(embinplist)
proj = Dense(32,activation = "relu",use_bias = True)(combine)
proj = Dense(1,activation = "sigmoid")(proj)

emb = Model(inplist,proj)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
emb.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          16512       input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          16512       input_2[0][0]                    
______________________________________________________________________________________________

## probability

In [8]:
adam=Adam(lr=0.01,beta_1=0.9,beta_2=0.999,amsgrad=True)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
emb.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
history =emb.fit([n2v_start_train,n2v_end_train,lanes_train,oneway_train,roadtype_train,maxspeed_train,nodetype_train,lanes_df_train],
        congestion_train,
        epochs=100,batch_size=256,
        validation_data=([n2v_start_test,n2v_end_test,lanes_test,oneway_test,roadtype_test,maxspeed_test,nodetype_test,lanes_df_test],
        congestion_test)
       )

Train on 86992 samples, validate on 28998 samples
Epoch 1/100
86992/86992 [==============================] - 4s 47us/sample - loss: 0.3343 - acc: 0.8706 - val_loss: 0.2906 - val_acc: 0.8827
Epoch 2/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.2491 - acc: 0.9008 - val_loss: 0.2459 - val_acc: 0.9001
Epoch 3/100
86992/86992 [==============================] - 3s 38us/sample - loss: 0.1999 - acc: 0.9206 - val_loss: 0.2229 - val_acc: 0.9131
Epoch 4/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.1703 - acc: 0.9315 - val_loss: 0.2198 - val_acc: 0.9143
Epoch 5/100
86992/86992 [==============================] - 3s 40us/sample - loss: 0.1504 - acc: 0.9396 - val_loss: 0.2044 - val_acc: 0.9218
Epoch 6/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.1351 - acc: 0.9456 - val_loss: 0.2078 - val_acc: 0.9197
Epoch 7/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.1243 - acc: 0.9504 - val_l

Epoch 59/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.0286 - acc: 0.9887 - val_loss: 0.3410 - val_acc: 0.9370
Epoch 60/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.0283 - acc: 0.9892 - val_loss: 0.2960 - val_acc: 0.9375
Epoch 61/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.0297 - acc: 0.9892 - val_loss: 0.2711 - val_acc: 0.9380
Epoch 62/100
86992/86992 [==============================] - 3s 40us/sample - loss: 0.0271 - acc: 0.9897 - val_loss: 0.3311 - val_acc: 0.9371
Epoch 63/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.0307 - acc: 0.9884 - val_loss: 0.3465 - val_acc: 0.9348
Epoch 64/100
86992/86992 [==============================] - 3s 40us/sample - loss: 0.0303 - acc: 0.9887 - val_loss: 0.3284 - val_acc: 0.9397
Epoch 65/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.0271 - acc: 0.9894 - val_loss: 0.3448 - val_acc: 0.9376
Epoch 66/100


In [49]:
emb.save('/home/fyp2/Ideaer/Madrid/model/all')

In [10]:
loss_hist = history.history['acc']
acc_history = history.history["val_acc"]
print("max acc :"+str(np.max(loss_hist)))
print("max validation accuarcy :"+str(np.max(acc_history)))

max acc :0.99294186
max validation accuarcy :0.9399614


In [82]:
i=0
##[n2v_start_test[i],n2v_end_test[i],lanes_test[i],oneway_test[i],roadtype_test[i],maxspeed_test[i],nodetype_test[i],lanes_df_test[i]]
emb.predict([n2v_start_test,n2v_end_test,lanes_test,oneway_test,roadtype_test,maxspeed_test,nodetype_test,lanes_df_test])

array([[0.0000000e+00],
       [3.1888485e-06],
       [1.0848045e-05],
       ...,
       [9.9795830e-01],
       [0.0000000e+00],
       [0.0000000e+00]], dtype=float32)

In [75]:
i=0
test=[n2v_start_test[i],n2v_end_test[i],lanes_test[i],oneway_test[i],roadtype_test[i],maxspeed_test[i],nodetype_test[i],lanes_df_test[i]]
test

[array([ 1.3915808e+00, -1.2021008e+00, -8.6221635e-01, -5.3120440e-01,
        -1.2984003e+00,  1.0230322e+00,  4.2808083e-01,  2.0948966e+00,
        -2.4878111e-01, -5.8579224e-01, -5.7401049e-01, -8.0279458e-01,
         1.8981482e-01, -1.4431199e+00, -1.5599100e-01,  1.0038726e+00,
         1.1355623e+00, -2.0026465e-01,  4.1419941e-01,  1.7009690e+00,
         6.9353431e-01, -1.2898475e-01, -5.1860315e-01, -8.9238323e-02,
         3.7774056e-01,  7.1910536e-01,  5.4402542e-01,  1.6002256e+00,
        -1.0757015e+00,  1.1525217e-01, -1.6179285e+00, -4.1993561e-01,
         3.6505306e-01, -2.1268131e-01,  8.0536032e-01,  9.0666682e-01,
        -5.4495521e-02,  2.5978827e-01, -1.4340067e+00, -1.3625052e+00,
        -2.4512237e-01,  9.5161623e-01,  8.8415638e-02,  5.2044123e-01,
         2.9327336e-01,  8.8279331e-01, -1.6011559e+00,  9.7170585e-01,
        -1.0048312e-01, -1.3242556e+00, -1.3006489e-01, -9.6572004e-02,
         4.0462214e-01,  1.0023009e+00,  1.4971946e+00, -1.02900

In [70]:
[n2v_start_train,n2v_end_train,lanes_train,oneway_train,roadtype_train,maxspeed_train,nodetype_train,lanes_df_train]

[array([[-0.37947005,  0.26185176,  0.47902697, ..., -0.7498107 ,
         -0.4113174 , -0.249935  ],
        [-0.50106674, -0.7048724 ,  0.31324285, ..., -1.2957523 ,
         -1.3448924 , -0.1651432 ],
        [-0.683296  , -0.17272215,  0.76008177, ..., -0.9627307 ,
         -0.90867686, -0.10340204],
        ...,
        [-0.3952649 , -0.01194038, -1.5359766 , ...,  1.2544329 ,
          1.2756613 ,  1.1548898 ],
        [-0.11672781,  0.61825126,  0.15074334, ...,  0.35550815,
         -0.80549353,  0.54629356],
        [-0.10544737,  0.15929723, -0.03256911, ..., -1.2606261 ,
         -1.2704748 ,  1.2572038 ]], dtype=float32),
 array([[-0.38298073,  0.31224012,  0.5369416 , ..., -1.099349  ,
         -0.8188122 , -0.38197225],
        [-0.52174056, -0.6799312 ,  0.14759552, ..., -1.154672  ,
         -1.314951  , -0.17409125],
        [-0.4240602 , -0.14925338,  0.87361354, ..., -1.3616533 ,
         -0.54464585, -0.33303022],
        ...,
        [-0.10554776, -0.06807904, -0.8

In [81]:
n2v_start_test[0]

array([ 1.3915808e+00, -1.2021008e+00, -8.6221635e-01, -5.3120440e-01,
       -1.2984003e+00,  1.0230322e+00,  4.2808083e-01,  2.0948966e+00,
       -2.4878111e-01, -5.8579224e-01, -5.7401049e-01, -8.0279458e-01,
        1.8981482e-01, -1.4431199e+00, -1.5599100e-01,  1.0038726e+00,
        1.1355623e+00, -2.0026465e-01,  4.1419941e-01,  1.7009690e+00,
        6.9353431e-01, -1.2898475e-01, -5.1860315e-01, -8.9238323e-02,
        3.7774056e-01,  7.1910536e-01,  5.4402542e-01,  1.6002256e+00,
       -1.0757015e+00,  1.1525217e-01, -1.6179285e+00, -4.1993561e-01,
        3.6505306e-01, -2.1268131e-01,  8.0536032e-01,  9.0666682e-01,
       -5.4495521e-02,  2.5978827e-01, -1.4340067e+00, -1.3625052e+00,
       -2.4512237e-01,  9.5161623e-01,  8.8415638e-02,  5.2044123e-01,
        2.9327336e-01,  8.8279331e-01, -1.6011559e+00,  9.7170585e-01,
       -1.0048312e-01, -1.3242556e+00, -1.3006489e-01, -9.6572004e-02,
        4.0462214e-01,  1.0023009e+00,  1.4971946e+00, -1.0290091e-02,
      

In [79]:
n2v_start_test[0].shape

(128,)